In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ===============================
# 1. User Settings (Parameters and Options)
# ===============================
# Data period to analyze (last N days)
N = 250  # modify as needed

# File path (testing with one symbol)
file_path = '/content/drive/MyDrive/Data/FNC/Historical_Data/GLD.csv'

# -------------------------------
# [Long System 1] L1 and Pyramiding Options
# -------------------------------
use_L1 = True                   # Use Long System 1
use_L1_pyramid = True           # Use pyramiding for Long System 1
L1a = 20                        # Entry: Breakout of highest high over last L1a days
L1b = 10                        # Exit: Price falls below lowest low over last L1b days after entry
L1c = 2.0                       # Exit Rule 1: (L1c × ATR)
L1d = 20                        # ATR period for Exit Rule 1
L1Pa = 0.5                      # Pyramiding additional entry: coefficient
L1Pb = 20                       # ATR period for pyramiding
L1Pc = 3                        # Maximum pyramiding count

# -------------------------------
# [Long System 2] L2 and Pyramiding Options
# -------------------------------
use_L2 = True                   # Use Long System 2
use_L2_pyramid = True           # Use pyramiding for Long System 2
L2a = 55                        # Entry: Breakout of highest high over last L2a days
L2b = 20                        # Exit: Price falls below lowest low over last L2b days after entry
L2c = 2.0                       # Exit Rule 1: (L2c × ATR)
L2d = 20                        # ATR period for Exit Rule 1
L2Pa = 0.5                      # Pyramiding additional entry: coefficient
L2Pb = 20                       # ATR period for pyramiding
L2Pc = 3                        # Maximum pyramiding count

# -------------------------------
# [Short System 1] S1 and Pyramiding Options
# -------------------------------
use_S1 = True                   # Use Short System 1
use_S1_pyramid = True           # Use pyramiding for Short System 1
S1a = 20                        # Entry: Breakout below the lowest low over last S1a days
S1b = 10                        # Exit: Price rises above highest high over last S1b days after entry
S1c = 2.0                       # Exit Rule 1: (S1c × ATR)
S1d = 20                        # ATR period for Exit Rule 1
S1Pa = 0.5                      # Pyramiding additional entry: coefficient
S1Pb = 20                       # ATR period for pyramiding
S1Pc = 3                        # Maximum pyramiding count

# -------------------------------
# [Short System 2] S2 and Pyramiding Options
# -------------------------------
use_S2 = True                   # Use Short System 2
use_S2_pyramid = True           # Use pyramiding for Short System 2
S2a = 55                        # Entry: Breakout below the lowest low over last S2a days
S2b = 20                        # Exit: Price rises above highest high over last S2b days after entry
S2c = 2.0                       # Exit Rule 1: (S2c × ATR)
S2d = 20                        # ATR period for Exit Rule 1
S2Pa = 0.5                     # Pyramiding additional entry: coefficient
S2Pb = 20                      # ATR period for pyramiding
S2Pc = 3                       # Maximum pyramiding count

# -------------------------------
# Choose Exit Rule (1 or 2) for Long/Short separately (default is Rule 1)
exit_rule_long = 1  # 1: Exit Rule 1 (ATR-based from entry price), 2: Based on recent L?b lowest/highest price
exit_rule_short = 1

# -------------------------------
# Run Backtest?
use_backtest = True

# ===============================
# Helper Function: Format Date (YYYY-MM-DD)
# ===============================
def format_date(date_val):
    return pd.to_datetime(date_val).strftime('%Y-%m-%d')

# ===============================
# 2. ATR (Average True Range) Calculation Function
# ===============================
def ATR(data, period):
    high = data['High']
    low = data['Low']
    close = data['Close']
    prev_close = close.shift(1)
    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window=period, min_periods=1).mean()
    return atr

# ===============================
# 3. Data Loading and Preprocessing
# ===============================
df = pd.read_csv(file_path)
df.sort_values('Date', inplace=True)
# Use only the last N days of data
df = df.tail(N).reset_index(drop=True)

# ATR calculation (create separate ATR columns for each system's period)
df['ATR_L1Pb'] = ATR(df, L1Pb)
df['ATR_L1d'] = ATR(df, L1d)
df['ATR_L2Pb'] = ATR(df, L2Pb)
df['ATR_L2d'] = ATR(df, L2d)
df['ATR_S1Pb'] = ATR(df, S1Pb)
df['ATR_S1d'] = ATR(df, S1d)
df['ATR_S2Pb'] = ATR(df, S2Pb)
df['ATR_S2d'] = ATR(df, S2d)

# ===============================
# 4. Simulation Functions for Each System
# ===============================
# --- Long System 1 (L1) Simulation ---
def simulate_long_system(df, entry_lb, exit_lb, exit_mult, use_pyramid, pyramid_mult, max_pyramid):
    trades = []     # Completed trades
    position = None # Position: {'entries': [(date, price), ...], 'entry_index': first entry index}
    last_entry_price = None
    last_entry_atr = None   # ATR at last entry
    pyramid_count = 0

    for i in range(entry_lb, len(df)):
        row = df.iloc[i]
        date = row['Date']
        close_price = row['Close']
        # Entry condition when no position exists
        if position is None:
            if close_price > df.iloc[i-entry_lb:i]['High'].max():
                position = {'entries': [(date, close_price)], 'entry_index': i}
                last_entry_price = close_price
                last_entry_atr = row['ATR_L1d']
                pyramid_count = 0
        else:
            # Pyramiding additional entry condition (threshold: last entry price + (pyramid coefficient × ATR_L1Pb))
            if use_pyramid and pyramid_count < max_pyramid:
                threshold = last_entry_price + (pyramid_mult * row['ATR_L1Pb'])
                if close_price > threshold:
                    position['entries'].append((date, close_price))
                    last_entry_price = close_price
                    last_entry_atr = row['ATR_L1d']
                    pyramid_count += 1
            # Exit conditions
            exit_flag = False
            if close_price < df.iloc[i-exit_lb:i]['Low'].min():
                exit_flag = True
            elif exit_rule_long == 1:
                threshold_exit = last_entry_price - (exit_mult * last_entry_atr)
                if close_price < threshold_exit:
                    exit_flag = True
            if exit_flag:
                entries = position['entries']
                first_entry_price = entries[0][1]
                profit_pct = (close_price / first_entry_price - 1) * 100
                trades.append({
                    'entries': entries,
                    'entry_date': entries[0][0],
                    'entry_index': position['entry_index'],
                    'entry_price': first_entry_price,
                    'exit_date': date,
                    'exit_index': i,
                    'exit_price': close_price,
                    'profit_pct': profit_pct,
                    'system': 'L1'
                })
                position = None
                last_entry_price = None
                last_entry_atr = None
                pyramid_count = 0

    open_trade = None
    if position is not None:
        entries = position['entries']
        first_entry_price = entries[0][1]
        exit_threshold = last_entry_price - (exit_mult * last_entry_atr) if exit_rule_long==1 else None
        open_trade = {
            'entries': entries,
            'entry_date': entries[0][0],
            'entry_index': position['entry_index'],
            'entry_price': first_entry_price,
            'current_date': df.iloc[-1]['Date'],
            'current_price': df.iloc[-1]['Close'],
            'profit_pct': (df.iloc[-1]['Close'] / first_entry_price - 1) * 100,
            'exit_threshold': exit_threshold,
            'system': 'L1'
        }
    return trades, open_trade

# --- Long System 2 (L2) Simulation ---
def simulate_long_system2(df, entry_lb, exit_lb, exit_mult, use_pyramid, pyramid_mult, max_pyramid):
    trades = []
    position = None
    last_entry_price = None
    last_entry_atr = None
    pyramid_count = 0

    for i in range(entry_lb, len(df)):
        row = df.iloc[i]
        date = row['Date']
        close_price = row['Close']
        if position is None:
            if close_price > df.iloc[i-entry_lb:i]['High'].max():
                position = {'entries': [(date, close_price)], 'entry_index': i}
                last_entry_price = close_price
                last_entry_atr = row['ATR_L2d']
                pyramid_count = 0
        else:
            if use_pyramid and pyramid_count < max_pyramid:
                threshold = last_entry_price + (pyramid_mult * row['ATR_L2Pb'])
                if close_price > threshold:
                    position['entries'].append((date, close_price))
                    last_entry_price = close_price
                    last_entry_atr = row['ATR_L2d']
                    pyramid_count += 1
            exit_flag = False
            if close_price < df.iloc[i-exit_lb:i]['Low'].min():
                exit_flag = True
            elif exit_rule_long == 1:
                threshold_exit = last_entry_price - (exit_mult * last_entry_atr)
                if close_price < threshold_exit:
                    exit_flag = True
            if exit_flag:
                entries = position['entries']
                first_entry_price = entries[0][1]
                profit_pct = (close_price / first_entry_price - 1) * 100
                trades.append({
                    'entries': entries,
                    'entry_date': entries[0][0],
                    'entry_index': position['entry_index'],
                    'entry_price': first_entry_price,
                    'exit_date': date,
                    'exit_index': i,
                    'exit_price': close_price,
                    'profit_pct': profit_pct,
                    'system': 'L2'
                })
                position = None
                last_entry_price = None
                last_entry_atr = None
                pyramid_count = 0

    open_trade = None
    if position is not None:
        entries = position['entries']
        first_entry_price = entries[0][1]
        exit_threshold = last_entry_price - (exit_mult * last_entry_atr) if exit_rule_long==1 else None
        open_trade = {
            'entries': entries,
            'entry_date': entries[0][0],
            'entry_index': position['entry_index'],
            'entry_price': first_entry_price,
            'current_date': df.iloc[-1]['Date'],
            'current_price': df.iloc[-1]['Close'],
            'profit_pct': (df.iloc[-1]['Close'] / first_entry_price - 1) * 100,
            'exit_threshold': exit_threshold,
            'system': 'L2'
        }
    return trades, open_trade

# --- Short System 1 (S1) Simulation ---
def simulate_short_system(df, entry_lb, exit_lb, exit_mult, use_pyramid, pyramid_mult, max_pyramid):
    trades = []
    position = None
    last_entry_price = None
    last_entry_atr = None
    pyramid_count = 0

    for i in range(entry_lb, len(df)):
        row = df.iloc[i]
        date = row['Date']
        close_price = row['Close']
        if position is None:
            if close_price < df.iloc[i-entry_lb:i]['Low'].min():
                position = {'entries': [(date, close_price)], 'entry_index': i}
                last_entry_price = close_price
                last_entry_atr = row['ATR_S1d']
                pyramid_count = 0
        else:
            if use_pyramid and pyramid_count < max_pyramid:
                threshold = last_entry_price - (pyramid_mult * row['ATR_S1Pb'])
                if close_price < threshold:
                    position['entries'].append((date, close_price))
                    last_entry_price = close_price
                    last_entry_atr = row['ATR_S1d']
                    pyramid_count += 1
            exit_flag = False
            if close_price > df.iloc[i-exit_lb:i]['High'].max():
                exit_flag = True
            elif exit_rule_short == 1:
                threshold_exit = last_entry_price + (exit_mult * last_entry_atr)
                if close_price > threshold_exit:
                    exit_flag = True
            if exit_flag:
                entries = position['entries']
                first_entry_price = entries[0][1]
                profit_pct = (first_entry_price / close_price - 1) * 100
                trades.append({
                    'entries': entries,
                    'entry_date': entries[0][0],
                    'entry_index': position['entry_index'],
                    'entry_price': first_entry_price,
                    'exit_date': date,
                    'exit_index': i,
                    'exit_price': close_price,
                    'profit_pct': profit_pct,
                    'system': 'S1'
                })
                position = None
                last_entry_price = None
                last_entry_atr = None
                pyramid_count = 0

    open_trade = None
    if position is not None:
        entries = position['entries']
        first_entry_price = entries[0][1]
        exit_threshold = last_entry_price + (exit_mult * last_entry_atr) if exit_rule_short==1 else None
        open_trade = {
            'entries': entries,
            'entry_date': entries[0][0],
            'entry_index': position['entry_index'],
            'entry_price': first_entry_price,
            'current_date': df.iloc[-1]['Date'],
            'current_price': df.iloc[-1]['Close'],
            'profit_pct': (first_entry_price / df.iloc[-1]['Close'] - 1) * 100,
            'exit_threshold': exit_threshold,
            'system': 'S1'
        }
    return trades, open_trade

# --- Short System 2 (S2) Simulation ---
def simulate_short_system2(df, entry_lb, exit_lb, exit_mult, use_pyramid, pyramid_mult, max_pyramid):
    trades = []
    position = None
    last_entry_price = None
    last_entry_atr = None
    pyramid_count = 0

    for i in range(entry_lb, len(df)):
        row = df.iloc[i]
        date = row['Date']
        close_price = row['Close']
        if position is None:
            if close_price < df.iloc[i-entry_lb:i]['Low'].min():
                position = {'entries': [(date, close_price)], 'entry_index': i}
                last_entry_price = close_price
                last_entry_atr = row['ATR_S2d']
                pyramid_count = 0
        else:
            if use_pyramid and pyramid_count < max_pyramid:
                threshold = last_entry_price - (pyramid_mult * row['ATR_S2Pb'])
                if close_price < threshold:
                    position['entries'].append((date, close_price))
                    last_entry_price = close_price
                    last_entry_atr = row['ATR_S2d']
                    pyramid_count += 1
            exit_flag = False
            if close_price > df.iloc[i-exit_lb:i]['High'].max():
                exit_flag = True
            elif exit_rule_short == 1:
                threshold_exit = last_entry_price + (exit_mult * last_entry_atr)
                if close_price > threshold_exit:
                    exit_flag = True
            if exit_flag:
                entries = position['entries']
                first_entry_price = entries[0][1]
                profit_pct = (first_entry_price / close_price - 1) * 100
                trades.append({
                    'entries': entries,
                    'entry_date': entries[0][0],
                    'entry_index': position['entry_index'],
                    'entry_price': first_entry_price,
                    'exit_date': date,
                    'exit_index': i,
                    'exit_price': close_price,
                    'profit_pct': profit_pct,
                    'system': 'S2'
                })
                position = None
                last_entry_price = None
                last_entry_atr = None
                pyramid_count = 0

    open_trade = None
    if position is not None:
        entries = position['entries']
        first_entry_price = entries[0][1]
        exit_threshold = last_entry_price + (exit_mult * last_entry_atr) if exit_rule_short==1 else None
        open_trade = {
            'entries': entries,
            'entry_date': entries[0][0],
            'entry_index': position['entry_index'],
            'entry_price': first_entry_price,
            'current_date': df.iloc[-1]['Date'],
            'current_price': df.iloc[-1]['Close'],
            'profit_pct': (first_entry_price / df.iloc[-1]['Close'] - 1) * 100,
            'exit_threshold': exit_threshold,
            'system': 'S2'
        }
    return trades, open_trade

# ===============================
# 5. Run Simulations for Each System
# ===============================
results = {}

if use_L1:
    trades_L1, open_trade_L1 = simulate_long_system(df, L1a, L1b, L1c, use_L1_pyramid, L1Pa, L1Pc)
    results['L1'] = (trades_L1, open_trade_L1)
if use_L2:
    trades_L2, open_trade_L2 = simulate_long_system2(df, L2a, L2b, L2c, use_L2_pyramid, L2Pa, L2Pc)
    results['L2'] = (trades_L2, open_trade_L2)
if use_S1:
    trades_S1, open_trade_S1 = simulate_short_system(df, S1a, S1b, S1c, use_S1_pyramid, S1Pa, S1Pc)
    results['S1'] = (trades_S1, open_trade_S1)
if use_S2:
    trades_S2, open_trade_S2 = simulate_short_system2(df, S2a, S2b, S2c, use_S2_pyramid, S2Pa, S2Pc)
    results['S2'] = (trades_S2, open_trade_S2)

# ===============================
# 6. Backtest Results Calculation Function
# ===============================
def calc_results(df, trades, system_type):
    cumulative_return = 1.0
    trade_mdds = []
    for trade in trades:
        first_entry = trade['entry_price']
        exit_price = trade['exit_price']
        if system_type.startswith('S'):
            # For shorts: entry price / exit price
            trade_return = first_entry / exit_price - 1
        else:
            trade_return = exit_price / first_entry - 1
        cumulative_return *= (1 + trade_return)
        # Calculate MDD for the trade period
        entry_idx = trade['entry_index']
        exit_idx = trade['exit_index']
        trade_slice = df.iloc[entry_idx: exit_idx+1]
        high_price = trade_slice['High'].max()
        low_price = trade_slice['Low'].min()
        if system_type.startswith('S'):
            trade_mdd = (high_price - low_price) / low_price * 100 if low_price != 0 else 0
        else:
            trade_mdd = (high_price - low_price) / high_price * 100 if high_price != 0 else 0
        trade_mdds.append(trade_mdd)
    cumulative_return_pct = (cumulative_return - 1) * 100
    overall_mdd = max(trade_mdds) if trade_mdds else 0
    return cumulative_return_pct, overall_mdd

backtest_results = {}

if use_backtest:
    for system, (trades, open_trade) in results.items():
        if trades:
            cum_ret, mdd = calc_results(df, trades, system)
            backtest_results[system] = {'cumulative_return_pct': cum_ret, 'MDD_pct': mdd}
        else:
            backtest_results[system] = {'cumulative_return_pct': None, 'MDD_pct': None}

# ===============================
# 7. Output Results
# ===============================
# Backtest Period
start_date = format_date(df.iloc[0]['Date'])
end_date = format_date(df.iloc[-1]['Date'])
print("===== Backtest Period =====")
print("Start Date:", start_date, ", End Date:", end_date)

# Backtest Results
print("\n===== Backtest Results =====")
for system, result in backtest_results.items():
    if result['cumulative_return_pct'] is not None:
        print(f"{system}: Cumulative Return = {result['cumulative_return_pct']:.2f}%, MDD = {result['MDD_pct']:.2f}%")
    else:
        print(f"{system}: No Trades")

# Last Position Status by System (if position is open, show current position; if not, show last trade info)
print("\n===== Last Position Status by System =====")
for system, (trades, open_trade) in results.items():
    if open_trade is not None:
        first_entry_date = format_date(open_trade['entry_date'])
        if system in ['L1', 'L2']:
            print(f"{system} Position Open - Entry Date: {first_entry_date}, Entry Price: {open_trade['entry_price']:.2f}, Current Price: {open_trade['current_price']:.2f}")
            if len(open_trade['entries']) > 1:
                print("   Pyramiding Entry Details:")
                for idx, (entry_date, entry_price) in enumerate(open_trade['entries']):
                    print(f"      Entry {idx+1}: Date: {format_date(entry_date)}, Price: {entry_price:.2f}")
            if exit_rule_long == 1 and open_trade['exit_threshold'] is not None:
                pct_move = (open_trade['entry_price'] - open_trade['exit_threshold']) / open_trade['entry_price'] * 100
                print(f"  >> Exit Rule Applied: Exit if price drops by {pct_move:.2f}% from entry price")
        else:
            print(f"{system} Position Open - Entry Date: {first_entry_date}, Entry Price: {open_trade['entry_price']:.2f}, Current Price: {open_trade['current_price']:.2f}")
            if len(open_trade['entries']) > 1:
                print("   Pyramiding Entry Details:")
                for idx, (entry_date, entry_price) in enumerate(open_trade['entries']):
                    print(f"      Entry {idx+1}: Date: {format_date(entry_date)}, Price: {entry_price:.2f}")
            if exit_rule_short == 1 and open_trade['exit_threshold'] is not None:
                pct_move = (open_trade['exit_threshold'] - open_trade['entry_price']) / open_trade['entry_price'] * 100
                print(f"  >> Exit Rule Applied: Exit if price rises by {pct_move:.2f}% from entry price")
    else:
        if trades:
            last_trade = trades[-1]
            print(f"{system} Last Trade - Entry Date: {format_date(last_trade['entry_date'])}, Entry Price: {last_trade['entry_price']:.2f}, Exit Date: {format_date(last_trade['exit_date'])}, Exit Price: {last_trade['exit_price']:.2f}, Return: {last_trade['profit_pct']:.2f}%")
            if len(last_trade['entries']) > 1:
                print("   Pyramiding Entry Details:")
                for idx, (entry_date, entry_price) in enumerate(last_trade['entries']):
                    print(f"      Entry {idx+1}: Date: {format_date(entry_date)}, Price: {entry_price:.2f}")
        else:
            print(f"{system}: No Trades")

# Final Trade Details and Pyramiding Entry Information by System
print("\n===== Final Trade Details and Pyramiding Entry Information by System =====")
for system, (trades, open_trade) in results.items():
    if trades:
        for trade in trades:
            print(f"{system} Trade - Entry Date: {format_date(trade['entry_date'])}, Entry Price: {trade['entry_price']:.2f}, Exit Date: {format_date(trade['exit_date'])}, Exit Price: {trade['exit_price']:.2f}, Return: {trade['profit_pct']:.2f}%")
            if len(trade['entries']) > 1:
                print("   Pyramiding Entry Details:")
                for idx, (entry_date, entry_price) in enumerate(trade['entries']):
                    print(f"      Entry {idx+1}: Date: {format_date(entry_date)}, Price: {entry_price:.2f}")
    else:
        print(f"{system}: No Trades")

# # ===============================
# # 8. Output: Exit Signal on Last Day
# # ===============================
# last_day = format_date(df.iloc[-1]['Date'])
# print("\n===== Last Day Exit Signals =====")
# for system, (trades, open_trade) in results.items():
#     # Filter trades that exited on the last day
#     exit_trades = [trade for trade in trades if format_date(trade['exit_date']) == last_day]
#     if exit_trades:
#         for trade in exit_trades:
#             print(f"{system} Exit Signal on {last_day} - Entry Price: {trade['entry_price']:.2f}, Exit Price: {trade['exit_price']:.2f}, Return: {trade['profit_pct']:.2f}%")
#     else:
#         print(f"{system}: No exit signal on last day")

# # ===============================
# # 9. Output: Last Entry Signal for Open Positions (if any)
# # ===============================
# print("\n===== Last Entry Signal for Open Positions =====")
# for system, (trades, open_trade) in results.items():
#     if open_trade is not None:
#         last_entry_date, last_entry_price = open_trade['entries'][-1]
#         print(f"{system} Open Position - Last Entry Signal: Date: {format_date(last_entry_date)}, Price: {last_entry_price:.2f}")
#     else:
#         print(f"{system}: No open position")

# ===============================
# 7. New Output Sections: Binary Signals
# ===============================
last_day_str = format_date(df.iloc[-1]['Date'])

# (1) Last Day Exit Signals: 1 if any trade exited on the last day, else 0.
print("\n===== Last Day Exit Signals ===== (1 if an exit signal occurred on the last day, else 0)")
for system, (trades, open_trade) in results.items():
    flag_exit = 0
    for trade in trades:
        if format_date(trade['exit_date']) == last_day_str:
            flag_exit = 1
            break
    print(f"{system}: {flag_exit}")

# (2) Last Day Open Signals: 1 if any entry signal occurred on the last day, else 0.
#     (Check if any completed trade or an open_trade has an entry on last day)
print("\n===== Last Day Open Signals ===== (1 if an entry signal occurred on the last day, else 0)")
for system, (trades, open_trade) in results.items():
    flag_open = 0
    for trade in trades:
        if format_date(trade['entry_date']) == last_day_str:
            flag_open = 1
            break
    if open_trade is not None:
        # Check the last entry in the open position
        if format_date(open_trade['entries'][-1][0]) == last_day_str:
            flag_open = 1
    print(f"{system}: {flag_open}")

# (3) Open Positions: 1 if there is an open position, else 0.
print("\n===== Open Positions ===== (1 if a position is currently open, else 0)")
for system, (trades, open_trade) in results.items():
    flag_position = 1 if open_trade is not None else 0
    print(f"{system}: {flag_position}")

===== Backtest Period =====
Start Date: 2024-04-02 , End Date: 2025-03-31

===== Backtest Results =====
L1: Cumulative Return = 2.85%, MDD = 8.51%
L2: Cumulative Return = 1.38%, MDD = 11.43%
S1: Cumulative Return = -6.40%, MDD = 3.85%
S2: No Trades

===== Last Position Status by System =====
L1 Position Open - Entry Date: 2025-03-13, Entry Price: 275.13, Current Price: 288.14
   Pyramiding Entry Details:
      Entry 1: Date: 2025-03-13, Price: 275.13
      Entry 2: Date: 2025-03-17, Price: 276.73
      Entry 3: Date: 2025-03-18, Price: 279.96
      Entry 4: Date: 2025-03-27, Price: 281.97
  >> Exit Rule Applied: Exit if price drops by -0.46% from entry price
L2 Position Open - Entry Date: 2025-01-24, Entry Price: 255.65, Current Price: 288.14
   Pyramiding Entry Details:
      Entry 1: Date: 2025-01-24, Price: 255.65
      Entry 2: Date: 2025-01-30, Price: 258.05
      Entry 3: Date: 2025-02-03, Price: 259.94
      Entry 4: Date: 2025-02-04, Price: 262.50
  >> Exit Rule Applied: Exit i